## Load Env Vars

In [1]:
from dotenv import load_dotenv
import os


# Load the environment variables from the .env file
load_dotenv()

os.environ["LANGCHAIN_PROJECT"] = "Multi-Agents-Plan-Execute"


## Web Searching Tool


In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools import DuckDuckGoSearchResults


# For Debugging
# tools = [DuckDuckGoSearchResults(max_results=10)]
tools = [TavilySearchResults(max_results=10)]

## Create Agent

In [ ]:
from langchain import hub
from langchain_openai import AzureChatOpenAI

from langgraph.prebuilt import create_react_agent


prompt = hub.pull("wfh/react-agent-executor")
prompt.pretty_print()


deployment_name = 'gpt-4o-mini'

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=deployment_name,
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

agent_executor = create_react_agent(llm, tools, messages_modifier=prompt)

Test a simple call

In [ ]:
agent_executor.invoke(
    {
        "messages": [("user", "best places to visit in beijing")]
    }
)

## Define PlanExecute State

In [5]:
import operator
from typing import Annotated, List, Tuple, TypedDict


class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

## Planning Step

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field


class Plan(BaseModel):
    """Plan to follow in future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

In [7]:
from langchain_core.prompts import ChatPromptTemplate


planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
                For the given objective, come up with a simple step by step plan. \
                This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
                The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.
            """,
        ),
        ("placeholder", "{messages}"),
    ]
)


planner = planner_prompt | AzureChatOpenAI(
    temperature=0,
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=deployment_name,
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
).with_structured_output(Plan)

In [ ]:
plan = planner.invoke(
    {
        "messages": [
            ("user", "According to most famous travel guides, which is the top 10 places to visit in Beijing, China?")
        ]
    }
)

for step in plan.steps:
    print(step)

## Re-Plan Step


In [9]:
from typing import Union


class Response(BaseModel):
    """Response to user."""

    response: str


class Act(BaseModel):
    """Action to perform."""

    action: Union[Response, Plan] = Field(
        description="Action to perform. If you want to respond to user, use Response."
        "If you need to further use tools to get the answer, use Plan."
    )


replanner_prompt = ChatPromptTemplate.from_template("""
    For the given objective, come up with a simple step by step plan. 
    This plan should involve individual tasks, that if executed correctly will yield the correct answer. 
    Do not add any superfluous steps, keep it simple. 
    The result of the final step should be the final answer. 
    Make sure that each step has all the information needed - do not skip steps.

    Your objective was this:
    {input}

    Your original plan was this:
    {plan}

    You have currently done the follow steps:
    {past_steps}

    Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. 
    Otherwise, fill out the plan. It's preferable to return to the user instead of creating unecessary steps.
    Do not return previously done steps as part of the plan.
    """
)


replanner = replanner_prompt | AzureChatOpenAI(
    temperature=0,
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=deployment_name,
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
).with_structured_output(Act)

## Plan the Graph


In [10]:
from typing import Literal


async def execute_step(state: PlanExecute):
    print('state : ', state)

    plan = state["plan"]
    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    task_formatted = f"""For the following plan:
    {plan_str}\n\nYou are tasked with executing step {1}, {task}."""
    
    agent_response = await agent_executor.ainvoke(
        {"messages": [("user", task_formatted)]}
    )
    return {
        "past_steps": [(task, agent_response["messages"][-1].content)],
    }


async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"messages": [("user", state["input"])]})
    return {"plan": plan.steps}


async def replan_step(state: PlanExecute):
    output = await replanner.ainvoke(state)
    if isinstance(output.action, Response):
        return {"response": output.action.response}
    else:
        return {"plan": output.action.steps}


def should_end(state: PlanExecute) -> Literal["agent", "__end__"]:
    if "response" in state and state["response"]:
        return "__end__"
    else:
        return "agent"

In [11]:
from langgraph.graph import StateGraph, START


# Create Workflow
workflow = StateGraph(PlanExecute)

# Add the plan node
workflow.add_node("planner", plan_step)

# Add the execution step
workflow.add_node("agent", execute_step)

# Add a replan node
workflow.add_node("replan", replan_step)

workflow.add_edge(START, "planner")

# From plan we go to agent
workflow.add_edge("planner", "agent")

# From agent, we replan
workflow.add_edge("agent", "replan")

# Ending Workflow
workflow.add_conditional_edges(
    "replan",    
    should_end,
)


app = workflow.compile()

In [ ]:
from IPython.display import Image, display


display(Image(app.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
config = {"recursion_limit": 100}


inputs = {"input": "What are the top 5 best places to visit in Beijing, China? "}

async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

In [ ]:
print(v['response'])